# Статистичні методи
Індивідуальне завдання 2

Токарєв Павло, МФ-51

## Підготовка даних
Текст узятий звідсіля: https://pamiec.pl/pa/biblioteka-cyfrowa/publikacje/53461,Polska-pod-okupacja-19391945-tom-1.html

Для початку ми видаляємо усі непотрібні знаки та переноси.
Потім приводимо диакретичні символи до простих відповідників.

In [13]:
import math
import re
from typing import Dict, Iterable, List, Tuple
from collections import defaultdict
import unidecode as unidecode

bad_chars = "!(),-.0123456789:?[]\xad„”–"
with open("data/polish/1.txt") as file:
    text = file.read()
    text = "".join(c for c in text if c not in bad_chars).lower()
    text = unidecode.unidecode(re.sub(r"\s+", " ", text))
# sample
print(" ".join("\n"+word if i%20==19 else word for i,word in enumerate(text.split()) if i < 99))

ziemie polskie ktore w drugiej polowie wrzesnia r dostaly sie pod okupacje so wiecka nie byly po prostu polowa 
polski ktora tym tylko roznila sie od tej zagarnietej przez niemcow ze lezala na wschodzie i ze zajeli ja rosjanie 
byl to poniekad inny kraj juz przed wojna tereny polozone na wschod od wisly i bugu nazywano polska b a 
ziemie na zachod od linii obu tych rzek polska a byla to zatem gorsza polska polska drugiej jakosci skad brala 
sie ta pogarda dla kresow wschodnich ii rzeczypospolitej otoz ziemie te byly gospodarczo i cywilizacyjnie zacofane z dominujacym rolnictwem ponad


Усі літери тексту.

In [14]:
chars = sorted(set(text) - set(" "))
chars

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z']

Усі слова тексту.

In [15]:
words = [word for word in text.split() if len(word) > 1]
words[:20]

['ziemie',
 'polskie',
 'ktore',
 'drugiej',
 'polowie',
 'wrzesnia',
 'dostaly',
 'sie',
 'pod',
 'okupacje',
 'so',
 'wiecka',
 'nie',
 'byly',
 'po',
 'prostu',
 'polowa',
 'polski',
 'ktora',
 'tym']

Кількість слів.

In [16]:
len(words)

1491

Функція розбиття на n-грами.

In [17]:
def ngrams(words: Iterable[str], n: int) -> Dict[str, int]:
    d: Dict[str, int] = defaultdict(int)
    for word in words:
        if len(word) < n:
            continue
        for i in range(len(word)-(n-1)):
            d[word[i:i+n]] += 1
    return d

Отримані n-грами.

In [18]:
twograms = ngrams(words, 2)
list(twograms.items())[:20]

[('zi', 44),
 ('ie', 337),
 ('em', 66),
 ('mi', 106),
 ('po', 123),
 ('ol', 73),
 ('ls', 23),
 ('sk', 56),
 ('ki', 79),
 ('kt', 28),
 ('to', 66),
 ('or', 62),
 ('re', 48),
 ('dr', 6),
 ('ru', 19),
 ('ug', 6),
 ('gi', 13),
 ('ej', 74),
 ('lo', 63),
 ('ow', 174)]

In [19]:
trigrams = ngrams(words, 3)
list(trigrams.items())[:20]

[('zie', 31),
 ('iem', 26),
 ('emi', 7),
 ('mie', 31),
 ('pol', 42),
 ('ols', 23),
 ('lsk', 20),
 ('ski', 29),
 ('kie', 36),
 ('kto', 21),
 ('tor', 27),
 ('ore', 9),
 ('dru', 3),
 ('rug', 3),
 ('ugi', 5),
 ('gie', 6),
 ('iej', 27),
 ('olo', 10),
 ('low', 15),
 ('owi', 47)]

Функції для обчислення матриці переходу та прогностичної здатності.

In [20]:
from collections import OrderedDict
from pandas import DataFrame

def prognozeability(matrix:List[Tuple[str, List[float]]]):
    pm = [
        (k, [1+sum(map(lambda p: p*math.log(p,2.0) if p > 0 else 0, l))/math.log(len(l), 2.0)])
        for k, l in matrix
    ]
    return DataFrame.from_dict(OrderedDict(pm), orient="index", columns=["prog. ability"])

def matrix(ngrams: Dict[str, int]):
    nmgram_amount = defaultdict(int)
    for k,v in ngrams.items():
        nmgram_amount[k[:-1]]+=v
    rev_ind = list(enumerate(sorted({key[:-1] for key in ngrams.keys()})))
    indexes = {key:i for i,key in rev_ind}
    m = [(rev_ind[i], [0 for _ in range(len(chars))]) for i in range(len(rev_ind))]
    for k,v in ngrams.items():
        c = k[-1]
        k = k[:-1]
        i = indexes[k]
        a = nmgram_amount[k]
        m[i][1][chars.index(c)] = v/a
    
    return DataFrame.from_dict(OrderedDict(m), orient="index", columns=chars), prognozeability(m)

## Результати
### 2-грами

In [21]:
two_matrix, twopa = matrix(twograms)
two_matrix

,a,b,c,d,e,f,g,h,i,j,...,o,p,r,s,t,u,v,w,y,z
"(0, a)",0.001513,0.010590,0.167927,0.059002,0.000000,0.001513,0.022693,0.000000,0.009077,0.039334,...,0.000000,0.009077,0.083207,0.040847,0.046899,0.013616,0.000000,0.034796,0.000000,0.045386
"(1, b)",0.118056,0.000000,0.006944,0.000000,0.048611,0.000000,0.000000,0.000000,0.187500,0.000000,...,0.166667,0.000000,0.083333,0.013889,0.000000,0.034722,0.000000,0.000000,0.277778,0.000000
"(2, c)",0.035639,0.000000,0.006289,0.000000,0.069182,0.000000,0.000000,0.253669,0.161426,0.062893,...,0.052411,0.000000,0.000000,0.000000,0.006289,0.006289,0.000000,0.000000,0.077568,0.211740
"(3, d)",0.065041,0.020325,0.008130,0.000000,0.065041,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.223577,0.004065,0.024390,0.012195,0.000000,0.040650,0.000000,0.028455,0.048780,0.207317
"(4, e)",0.000000,0.009772,0.109121,0.065147,0.000000,0.000000,0.084691,0.000000,0.003257,0.120521,...,0.004886,0.019544,0.058632,0.099349,0.034202,0.000000,0.000000,0.027687,0.000000,0.048860
"(5, f)",0.266667,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.266667,0.000000,...,0.133333,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000
"(6, g)",0.163793,0.000000,0.000000,0.008621,0.000000,0.000000,0.000000,0.000000,0.112069,0.000000,...,0.448276,0.000000,0.112069,0.000000,0.000000,0.008621,0.000000,0.000000,0.000000,0.008621
"(7, h)",0.159091,0.000000,0.022727,0.000000,0.045455,0.000000,0.000000,0.000000,0.022727,0.000000,...,0.522727,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000
"(8, i)",0.142653,0.001427,0.088445,0.004280,0.480742,0.000000,0.001427,0.000000,0.011412,0.002853,...,0.034237,0.000000,0.000000,0.032810,0.022825,0.037090,0.000000,0.005706,0.000000,0.015692
"(9, j)",0.353659,0.012195,0.000000,0.006098,0.335366,0.000000,0.006098,0.000000,0.060976,0.000000,...,0.024390,0.000000,0.000000,0.073171,0.000000,0.042683,0.000000,0.000000,0.000000,0.000000


In [22]:
twopa

,prog. ability
"(0, a)",0.205033
"(1, b)",0.364175
"(2, c)",0.349077
"(3, d)",0.297276
"(4, e)",0.187629
"(5, f)",0.478753
"(6, g)",0.488442
"(7, h)",0.491809
"(8, i)",0.403523
"(9, j)",0.464106


### 3-грами

In [23]:
three_matrix, tripa = matrix(trigrams)
three_matrix

,a,b,c,d,e,f,g,h,i,j,...,o,p,r,s,t,u,v,w,y,z
"(0, aa)",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
"(1, ab)",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.285714,0.000000,...,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.0,0.000000,0.000000,0.000000
"(2, ac)",0.043011,0.000000,0.000000,0.000000,0.064516,0.0,0.000000,0.172043,0.021505,0.193548,...,0.075269,0.000000,0.000000,0.000000,0.010753,0.010753,0.0,0.000000,0.236559,0.161290
"(3, ad)",0.230769,0.000000,0.000000,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,...,0.269231,0.000000,0.038462,0.000000,0.000000,0.000000,0.0,0.038462,0.076923,0.153846
"(4, af)",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(322, zs)",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333
"(323, zt)",0.666667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.000000
"(324, zu)",0.000000,0.000000,0.300000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.100000,...,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
"(325, zw)",0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.700000,0.000000,...,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [24]:
tripa

,prog. ability
"(0, aa)",1.000000
"(1, ab)",0.512335
"(2, ac)",0.373335
"(3, ad)",0.410319
"(4, af)",1.000000
...,...
"(322, zs)",0.799716
"(323, zt)",0.799716
"(324, zu)",0.526506
"(325, zw)",0.747701
